In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DATA_PATH = os.path.join(os.getcwd(), '..', 'data')
os.listdir(DATA_PATH)

In [ ]:
demo_bank_df = pd.read_csv(os.path.join(DATA_PATH, '1. DEMO_BANK_INFO.csv'))
demo_bank_df.head()

In [ ]:
os.chdir(os.path.join(os.getcwd(), '..'))
os.getcwd()

In [ ]:
from src.utils import preprocessing_1
from src.utils import preprocessing_2
from src.utils import preprocessing_3

# Getting Data

In [ ]:
cleaned_demo_bank_df = preprocessing_1(demo_bank_df)

In [ ]:
cleaned_telco_df = preprocessing_2(DATA_PATH)

In [ ]:
full_df = preprocessing_3(cleaned_bank_df, cleaned_telco_df)
full_df.head()

# SVM

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
from sklearn import svm

In [ ]:
demo_bank_df_p1.info()

In [ ]:
scoring = ['precision_macro', 'recall_macro', 'accuracy']
clf = svm.SVC(kernel='rbf', C=1, random_state=0, probability=True)

In [ ]:
_x_train = demo_bank_df_p1.drop(columns = ['DELQ_FLG', 'loan_application_date'])
_x_train.drop(columns = list(_x_train.filter(regex='loan_application_date_DOW_*')), inplace=True)
x_train = _x_train.drop(columns = list(_x_train.filter(regex='loan_application_date_week_*')))

y_train = demo_bank_df_p1['DELQ_FLG']

print(x_train.shape, y_train.shape)

In [ ]:
x_train.columns

In [ ]:
scores = cross_validate(clf, x_train, y_train, scoring=scoring,
                        cv=5, return_train_score=False)
sorted(scores.keys())

In [ ]:
print("precision: {}, \nrecall: {} \naccuracy: {}".format(scores['test_precision_macro'], scores['test_recall_macro'], scores['test_accuracy']))

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

X_train, X_test, Y_train, Y_test = train_test_split(x_train, y_train, test_size=.2,
                                                    random_state=0)

clf.fit(X_train, Y_train)

In [ ]:
# Compute ROC curve and ROC area for each class
fpr, tpr, _ = roc_curve(Y_test, clf.predict_proba(X_test)[:,1], pos_label=2)
roc_auc = auc(fpr, tpr)
print(roc_auc)

In [ ]:
tpr

In [ ]:
#GRAPH DATA
plt.figure()
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.title('SVM Classifier ROC')
plt.plot(fpr, tpr, color='blue', lw=2, label='SVM ROC area = %0.2f)' % roc_auc)
plt.legend(loc="lower right")
plt.show()

# MLP

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [ ]:
def build_mlp_model(dims):
    '''
    build multi-layer perceptron network according to dims
    
    dims is an array of integers specifiying input, hidden and output layers
    '''
    model = Sequential()
    model.add(Dense(dims[1], activation = 'sigmoid', input_dim = dims[0]))
    for dim in range(1,len(dims) - 1):
        model.add(Dense(dims[dim+1], activation = 'sigmoid'))
    model.compile(loss=keras.losses.mean_squared_error, 
                  optimizer=keras.optimizers.Adam(lr=0.01),
                  metrics=['accuracy'])
    return model

In [ ]:
def train(model, epochs, batch_size, X_train, Y_train, X_test, Y_test):
    
    LOG_DIR = os.path.join(os.getcwd(), 'logs')
    tb = TensorBoard(LOG_DIR)
    weights_dir = 'weights/' + model.name + \
                '-{epoch:02d}-{loss:.2f}.hdf5'
    chkpt = ModelCheckpoint(filepath=weights_dir, monitor='loss', save_best_only=True, save_weights_only=True, mode='auto', period=1)
    early_stop = EarlyStopping(monitor='val_acc', min_delta=0, patience=5)
    
    history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, validation_data = (X_test, Y_test), callbacks=[tb, chkpt, early_stop])
    
    return history

In [ ]:
model_1 = build_mlp_model([50,100,200])

In [ ]:
train(model_1, epochs=50, )